In [2]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 110851 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.1-0ubuntu3~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.1-0ubuntu3~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.1-0ubuntu3~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=ht

In [1]:
!ls

sample_data


In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [4]:
cd drive/Houston

/content/drive/Houston


In [0]:
import numpy as np
import cv2

In [0]:
moment_hsi = []
for i in range(3):
    for j in range(4):
        img_path = "moment/pc_" + str(i) + "_" + str(j) + ".png"
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        moment_hsi.append(img)
        
moment_hsi = np.array(moment_hsi)

In [7]:
print(moment_hsi.shape)

(12, 349, 1905)


In [0]:
train_file_name = "labels/train.txt"
test_file_name = "labels/test.txt"

In [0]:
file = open(train_file_name)
triplets = file.read().split()
for i in range(0, len(triplets)):
    triplets[i] = triplets[i].split(",")
train_array = np.array(triplets, dtype=int)
file.close()


file = open(test_file_name)
triplets = file.read().split()
for i in range(0, len(triplets)):
    triplets[i] = triplets[i].split(",")
test_array = np.array(triplets, dtype=int)
file.close()

HEIGHT = train_array.shape[0]
WIDTH = train_array.shape[1]

In [0]:
moment_hsi_train_data = []
moment_hsi_test_data = []
train_labels = []
test_labels = []
for i in range(HEIGHT):
    for j in range(WIDTH):
        if train_array[i, j] != 0:
            moment_hsi_train_data.append(moment_hsi[:, i, j])
            train_labels.append(train_array[i, j])
        if test_array[i,j] != 0:
            moment_hsi_test_data.append(moment_hsi[:, i, j])
            test_labels.append(test_array[i, j])

In [0]:
moment_hsi_train_data = np.array(moment_hsi_train_data)
train_labels = np.array(train_labels)
test_labels = np.array(test_labels)

In [12]:
print(moment_hsi_train_data.shape)
print(train_labels.shape)
print(test_labels.shape)

(2832, 12)
(2832,)
(12197,)


In [13]:
import keras

train_one_hot = keras.utils.to_categorical(train_labels-1)
test_one_hot = keras.utils.to_categorical(test_labels-1)

Using TensorFlow backend.


In [0]:
import keras

train_one_hot = keras.utils.to_categorical(train_labels-1)
test_one_hot = keras.utils.to_categorical(test_labels-1)

In [0]:
HSI_PATCH_SIZE = 27
CONV1 = 500
CONV2 = 100
FC1 = 200
FC2 = 84
LEARNING_RATE = 0.005

In [16]:
padded_moment_hsi = np.lib.pad(moment_hsi, ((0,0), (HSI_PATCH_SIZE//2, HSI_PATCH_SIZE//2), (HSI_PATCH_SIZE//2,HSI_PATCH_SIZE//2)), 'reflect')
print(padded_moment_hsi.shape)

(12, 375, 1931)


In [0]:
def get_patches(data, patch_size, row, column):
  offset = patch_size // 2
  row_low = row - offset
  row_high = row + offset
  col_low = column - offset
  col_high = column + offset
  return data[0:, row_low:row_high + 1, col_low:col_high + 1].reshape(patch_size, patch_size, data.shape[0])

In [0]:
moment_hsi_train_patches = []
moment_hsi_test_patches = []
for i in range(HEIGHT):
  for j in range(WIDTH):
    if train_array[i, j] != 0:
      moment_hsi_train_patches.append(get_patches(padded_moment_hsi, HSI_PATCH_SIZE, i+HSI_PATCH_SIZE//2, j+HSI_PATCH_SIZE//2))
    if test_array[i, j] != 0:
      moment_hsi_test_patches.append(get_patches(padded_moment_hsi, HSI_PATCH_SIZE, i+HSI_PATCH_SIZE//2, j+HSI_PATCH_SIZE//2))

In [20]:
moment_hsi_train_patches = np.array(moment_hsi_train_patches)
moment_hsi_test_patches = np.array(moment_hsi_test_patches)
print(moment_hsi_train_patches.shape)
print(moment_hsi_test_patches.shape)

(2832, 27, 27, 12)
(12197, 27, 27, 12)


In [21]:
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Conv2D, Dense, Flatten
from tensorflow.python.keras.layers import InputLayer
from tensorflow.python.keras.layers import  MaxPooling2D
from tensorflow.python.keras.layers import BatchNormalization, Dropout
from tensorflow.python.keras.optimizers import Adam,SGD

BANDS = moment_hsi_train_patches.shape[3]
NUM_CLS = train_one_hot.shape[1]
BATCH_SIZE = 25

moment_hsi_model = Sequential()
moment_hsi_model.add(InputLayer(input_shape=(HSI_PATCH_SIZE, HSI_PATCH_SIZE, BANDS)))
moment_hsi_model.add(Conv2D(kernel_size=6, strides=2, filters=CONV1, padding='same', activation='relu', name='conv1'))
moment_hsi_model.add(BatchNormalization())
moment_hsi_model.add(MaxPooling2D(pool_size=2, strides=2))
moment_hsi_model.add(Conv2D(kernel_size=5, strides=2, filters=CONV2, padding='same', activation='relu', name='conv2'))
moment_hsi_model.add(BatchNormalization())
moment_hsi_model.add(MaxPooling2D(pool_size=2, strides=2))


moment_hsi_model.add(Flatten())
moment_hsi_model.add(Dense(FC1, activation='relu'))
moment_hsi_model.add(Dropout(0.6))
moment_hsi_model.add(Dense(FC2, activation='relu'))
moment_hsi_model.add(Dropout(0.4))
moment_hsi_model.add(Dense(NUM_CLS, activation='softmax'))

sgd = SGD(lr=LEARNING_RATE, decay=1e-6, momentum=0.9, nesterov=True)
moment_hsi_model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

moment_hsi_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1 (Conv2D)               (None, 14, 14, 500)       216500    
_________________________________________________________________
batch_normalization (BatchNo (None, 14, 14, 500)       2000      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 7, 7, 500)         0         
_________________________________________________________________
conv2 (Conv2D)               (None, 4, 4, 100)         1250100   
_________________________________________________________________
batch_normalization_1 (Batch (None, 4, 4, 100)         400       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 2, 2, 100)         0         
_________________________________________________________________
flatten (Flatten)            (None, 400)               0         
__________

In [56]:
history = moment_hsi_model.fit(moment_hsi_train_patches, train_one_hot, batch_size=BATCH_SIZE, shuffle=True, epochs=50)

Epoch 1/50
2832/2832 [==============================] - 2s 762us/step - loss: 0.0129 - acc: 0.9958
Epoch 2/50
2832/2832 [==============================] - 2s 734us/step - loss: 0.0105 - acc: 0.9968
Epoch 3/50
2832/2832 [==============================] - 2s 736us/step - loss: 0.0109 - acc: 0.9961
Epoch 4/50
2832/2832 [==============================] - 2s 738us/step - loss: 0.0130 - acc: 0.9961
Epoch 5/50
2832/2832 [==============================] - 2s 733us/step - loss: 0.0082 - acc: 0.9968
Epoch 6/50
2832/2832 [==============================] - 2s 735us/step - loss: 0.0220 - acc: 0.9944
Epoch 7/50
2832/2832 [==============================] - 2s 737us/step - loss: 0.0078 - acc: 0.9965
Epoch 8/50
2832/2832 [==============================] - 2s 734us/step - loss: 0.0113 - acc: 0.9958
Epoch 9/50
2832/2832 [==============================] - 2s 739us/step - loss: 0.0108 - acc: 0.9965
Epoch 10/50
2832/2832 [==============================] - 2s 729us/step - loss: 0.0114 - acc: 0.9961
Epoch 11/

In [0]:
from operator import truediv

def AA_andEachClassAccuracy(confusion_matrix):
    counter = confusion_matrix.shape[0]
    list_diag = np.diag(confusion_matrix)
    list_raw_sum = np.sum(confusion_matrix, axis=1)
    each_acc = np.nan_to_num(truediv(list_diag, list_raw_sum))
    average_acc = np.mean(each_acc)
    return each_acc, average_acc

In [57]:
test_cls = test_labels - 1
prediction = moment_hsi_model.predict(moment_hsi_test_patches).argmax(axis=-1)



from sklearn import metrics, preprocessing

overall_acc = metrics.accuracy_score(prediction, test_cls)
kappa = metrics.cohen_kappa_score(prediction, test_cls)
confusion_matrix = metrics.confusion_matrix(prediction, test_cls)
each_acc, average_acc = AA_andEachClassAccuracy(confusion_matrix) 


print("Overall Accuracy of training sapmles : ",overall_acc)
print("Average Accuracy of training samples : ",average_acc)
print("Kappa statistics of training samples : ",kappa)
print("Each class accuracy of training samples : ", each_acc)
print("Confusion matrix :", confusion_matrix)

Overall Accuracy of training sapmles :  0.6095761252767073
Average Accuracy of training samples :  0.620515287733426
Kappa statistics of training samples :  0.575568821210299
Each class accuracy of training samples :  [0.69595782 0.64673913 0.90588235 0.67030568 0.79115044 0.64285714
 0.72838428 0.32838155 0.59414226 0.63338789 0.5901034  0.51977401
 0.47181009 0.46064815 0.62820513]
Confusion matrix : [[792 122   0 128   0   1  21  14   0  24  29   2   0   0   5]
 [ 47 595   1 208   5   5  24   3  18   0   0   0   0   0  14]
 [  0   0 231   2   0   0   6  11   0   0   1   0   1   0   3]
 [ 35 157   0 614   0   0  73   2   0   0   0  23   0   0  12]
 [  0   1  26   2 894   0   0   2  33   0   0  62   0  35  75]
 [  1   0  43   3   0 108   3   7   0   0   1   1   0   0   1]
 [  0  25   4  23  10  23 834  19 142   5  17   2  39   0   2]
 [154 152  75  69  13   0  41 420  19 110  20 126   5   0  75]
 [  0   4   0   4  58   6  46  72 710  34  37   3   3   2 216]
 [  0   0  21   0   2   0  

In [0]:
moment_hsi_model.save_weights('Models/moment_hsi_model_weights.h5')